# Student Loan Risk with Deep Learning

In [2]:
!pip install tensorflow


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ed/1a/b4ab4b8f8b3a41fade4899fd00b5b2d2dad0981f3e1bb10df4c522975fd7/tensorflow-2.15.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa/39/5aae571e5a5f4de9c3445dae08a530498e5c53b0e74410eeeb0991c79047/gast-0.5.4-py3-none-any.whl.metadata
  Using cached google_pasta-0.2.0

  Obtaining dependency information for tensorboard-data-server<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/73/c6/825dab04195756cf8ff2e12698f22513b3db2f64925bdd41671bfb33aaa5/tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata
  Obtaining dependency information for cachetools<6.0,>=2.0.0 from https://files.pythonhosted.org/packages/fb/2b/a64c2d25a37aeb921fddb929111413049fc5f8b9a4c1aefaffaafe768d54/cachetools-5.3.3-py3-none-any.whl.metadata
  Obtaining dependency information for rsa<5,>=3.1.4 from https://files.pythonhosted.org/packages/49/97/fa78e3d2f65c02c8e1268b9aba606569fe97f6c8f7c2d74394553347c145/rsa-4.9-py3-none-any.whl.metadata
  Obtaining dependency information for requests-oauthlib>=0.7.0 from https://files.pythonhosted.org/packages/6f/bb/5deac77a9af870143c684ab46a7934038a53eb4aa975bc0687ed6ca2c610/requests_oauthlib-1.3.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for oauthlib>=3.0.0 from https://files.pythonhosted.org/

In [3]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

2024-02-27 18:17:40.861744: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 18:17:40.905975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 18:17:40.906012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 18:17:40.907242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 18:17:40.914272: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 18:17:40.934808: I tensorflow/core/platform/cpu_feature_guard.cc:1

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [4]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [7]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [8]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [9]:
# Define the target set y using the credit_ranking column


# Display a sample of y
# Assuming you have already imported pandas and performed the necessary data preprocessing steps

# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
print("Sample of target set (y):")
print(y.head())



Sample of target set (y):
0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64


In [12]:
# Define features set X by selecting all columns but credit_ranking


# Review the features DataFrame

# Assuming you have already imported pandas and performed the necessary data preprocessing steps

# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
print("Features set (X):")
print(X.head())



Features set (X):
   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0              7.4                0.70               0.00          1.9   
1              7.8                0.88               0.00          2.6   
2              7.8                0.76               0.04          2.3   
3             11.2                0.28               0.56          1.9   
4              7.4                0.70               0.00          1.9   

   alumni_success  study_major_code  time_to_completion  \
0           0.076              11.0                34.0   
1           0.098              25.0                67.0   
2           0.092              15.0                54.0   
3           0.075              17.0                60.0   
4           0.076              11.0                34.0   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                  0.9978            3.51              0.56   
1                  0.9968            3.20              0

### Step 3: Split the features and target sets into training and testing datasets.


In [13]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
from sklearn.model_selection import train_test_split

# Assuming X and y have been defined in the previous steps

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Display the shapes of the resulting datasets for verification
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)



Shape of X_train: (1279, 11)
Shape of X_test: (320, 11)
Shape of y_train: (1279,)
Shape of y_test: (320,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [16]:
# Create a StandardScaler instance


# Fit the scaler to the features training dataset


# Fit the scaler to the features training dataset
from sklearn.preprocessing import StandardScaler

# Assuming X_train and X_test have been defined in the previous steps

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Transform the features testing dataset using the scaler fitted to the training data
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [17]:
# Define the the number of inputs (features) to the model


# Review the number of features

# Assuming X_train_scaled has been defined in the previous steps

# Define the number of inputs (features) to the model
num_inputs = X_train_scaled.shape[1]

# Display the number of features
print("Number of features:", num_inputs)


Number of features: 11


In [19]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32

# Define the number of neurons in the output layer
output_nodes = 1


In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assuming num_inputs, hidden_nodes_layer1, hidden_nodes_layer2, and output_nodes have been defined

# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer with the specified number of nodes and ReLU activation function
model.add(Dense(units=hidden_nodes_layer1, activation='relu', input_dim=num_inputs))

# Add the second hidden layer with the specified number of nodes and ReLU activation function
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Add the output layer with the specified number of output neurons (for binary classification) and sigmoid activation function
model.add(Dense(units=output_nodes, activation='sigmoid'))




In [23]:
# Display the Sequential model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                768       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [25]:
# Compile the Sequential model
# Compile the Sequential model with binary_crossentropy loss function, adam optimizer, and accuracy metric
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [26]:
# Fit the model using 50 epochs and the training data
# Assuming X_train_scaled and y_train have been defined

# Fit the Sequential model to the training data for 50 epochs
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=1)


Epoch 1/50
40/40 [==============================] - 1s 3ms/step - loss: 0.6267 - accuracy: 0.6560
Epoch 2/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5396 - accuracy: 0.7467
Epoch 3/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5165 - accuracy: 0.7506
Epoch 4/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5040 - accuracy: 0.7553
Epoch 5/50
40/40 [==============================] - 0s 2ms/step - loss: 0.4961 - accuracy: 0.7553
Epoch 6/50
40/40 [==============================] - 0s 2ms/step - loss: 0.4901 - accuracy: 0.7631
Epoch 7/50
40/40 [==============================] - 0s 2ms/step - loss: 0.4850 - accuracy: 0.7639
Epoch 8/50
40/40 [==============================] - 0s 2ms/step - loss: 0.4794 - accuracy: 0.7639
Epoch 9/50
40/40 [==============================] - 0s 2ms/step - loss: 0.4750 - accuracy: 0.7654
Epoch 10/50
40/40 [==============================] - 0s 2ms/step - loss: 0.4713 - accuracy: 0.7694
Epoch 11/50
40/40 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [27]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data


# Display the model loss and accuracy results
# Assuming X_test_scaled and y_test have been defined

# Evaluate the model on the test data
evaluation_results = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print("\nEvaluation Results:")
print(f"Test Loss: {evaluation_results[0]:.4f}")
print(f"Test Accuracy: {evaluation_results[1]*100:.2f}%")



10/10 [==============================] - 0s 2ms/step - loss: 0.5560 - accuracy: 0.7344

Evaluation Results:
Test Loss: 0.5560
Test Accuracy: 73.44%


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [30]:
# Set the model's file path


# Export your model to a keras file

# Set the model's file path
model_file_path = "/home/antitheft/Documents/loans/student_loans.keras"

# Save the model to the specified file path
model.save(model_file_path)

# Display a message indicating that the model has been saved
print(f"Model has been saved to {model_file_path}")



Model has been saved to /home/antitheft/Documents/loans/student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [32]:
# Set the model's file path


# Load the model to a new object
from tensorflow.keras.models import load_model

# Set the model's file path
model_file_path = "/home/antitheft/Documents/loans/student_loans.keras"

# Load the model to a new object
loaded_model = load_model(model_file_path)



### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [33]:
# Make predictions with the test data


# Display a sample of the predictions

# Assuming X_test_scaled has been defined

# Make predictions on the test data
predictions = loaded_model.predict(X_test_scaled)

# Round the predictions to binary values (0 or 1)
rounded_predictions = [round(pred[0]) for pred in predictions]

# Display a sample of the predictions
sample_predictions = rounded_predictions[:10]  # Displaying the first 10 predictions as an example
print("Sample Predictions:", sample_predictions)


10/10 [==============================] - 0s 2ms/step
Sample Predictions: [0, 0, 1, 1, 1, 1, 1, 0, 1, 0]


In [34]:
# Save the predictions to a DataFrame and round the predictions to binary results
import pandas as pd

# Assuming y_test has been defined

# Create a DataFrame with actual and predicted values
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': rounded_predictions
})

# Display the DataFrame
print("Predictions DataFrame:")
print(predictions_df.head())


Predictions DataFrame:
      Actual  Predicted
75         0          0
1283       1          0
408        1          1
1281       1          1
1118       1          1


### Step 4: Display a classification report with the y test data and predictions

In [35]:
# Print the classification report with the y test data and predictions
from sklearn.metrics import classification_report

# Print the classification report with the y test data and predictions
print("Classification Report:\n")
print(classification_report(y_test, rounded_predictions))


Classification Report:

              precision    recall  f1-score   support

           0       0.74      0.69      0.71       154
           1       0.73      0.78      0.75       166

    accuracy                           0.73       320
   macro avg       0.74      0.73      0.73       320
weighted avg       0.73      0.73      0.73       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**



1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

To build a recommendation system for student loan options, relevant data would include:

Student Information: Personal details such as age, location, academic history, and field of study.
Financial Information: Income, credit history, and existing financial commitments.
Loan History: Any existing student loans, repayment history, and credit ranking.
Loan Options Data: Features and terms of available loan options, interest rates, and repayment plans.
Career Aspirations: Future career plans, expected income post-graduation.
This data is crucial for assessing a student's financial situation, creditworthiness, and preferences. By understanding their academic and career goals, the recommendation system can suggest loan options that align with their needs and financial capabilities.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

For a student loan recommendation system, a combination of collaborative and content-based filtering would be appropriate:

Collaborative Filtering: It considers the behavior and preferences of similar students to recommend loans. It can be effective when students with similar academic and financial backgrounds tend to make similar choices.

Content-Based Filtering: It leverages information about the students and loan options to make recommendations. This is valuable when considering a student's individual financial situation, academic history, and career aspirations.

The combination allows for a more personalized recommendation by considering both user behavior and the characteristics of loan options.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

Data Privacy and Security: Collecting sensitive information about students' financial situations raises concerns about data privacy and security. Ensuring robust security measures and obtaining informed consent from users is crucial to build trust and comply with regulations.

Dynamic Financial Situations: Financial situations of students can change rapidly, especially during their academic journey. Adapting to these changes, providing real-time updates, and offering flexible recommendations that align with the evolving financial context are challenges to address. Regular updates and feedback mechanisms would be necessary to keep recommendations relevant.